# Multiples Procesos por Nodo: Horovod

En tensorflow permite correr varios procesos por host de la siguente forma:

```python
from sagemaker.tensorflow import TensorFlow
tf_estimator = TensorFlow(entry_point='tf-train.py', role='SagemakerRole', 
                          train_instance_count=1, train_instance_type='ml.p2.xlarge',
                          framework_version='1.12', py_version='py3',
                          distributions= {
                             'mpi':{
                                 'enabled': True,
                                 'processes_per_host': 2, 
                                 'custom_mpi_options': '--NCCL_DEBUG INFO'
                             }
                          })
tf_estimator.fit('s3://bucket/path/to/training/data')
```

# Multiples Nodos

## Fully Replicated: se copia toda la data a todos los nodos

* Incrementa la estabilidad del model
* Datas no tan grandes
* Escala directamente el entrenamiento

```python
# cuando se configura el canal
{ 
    "ChannelName": "validation",
    "DataSource": {
        "S3DataSource": {
            "S3DataType": "S3Prefix",
            "S3Uri": "s3://bucket/path/to/training/data",
            "S3DataDistributionType": "FullyReplicated"
        }
    },
    "CompressionType": "None",
    "RecordWrapperType": "None"
}

```
___
## Sharded By S3 Key: se divide la data por grupos de s3 key prefix:

* Data mas grande
* Exponer parte del modelo a ejemplos especificos
* Entrenamiento más rápido

```python
# Paso 1: Dividir la data en pequeños Keys
# Paso 2: Crear S3 inputs
# Paso 3: Apuntar el S3 Input al directorio de S3

s3_train = s3_input(os.path.join(s3_input_path, 'train', TRAIN_DATA),
                    distribution='ShardedByS3Key', content_type='application/jsonlines')

```


___
## Parameter Server

Consolida el estado de los nodos en un estado compartido. Esta forma permite los algoritmos manejados de aws hacer el entrenamiento distribuido. 

### Usando Parameter Server con TensorFlow 

#### 1. Establecer el parámetro en Tensorflow

```python
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='tf-train.py', role='SagemakerRole', 
                          train_instance_count=1, train_instance_type='ml.p2.xlarge',
                          framework_version='1.11', py_version='py3',
                          distributions= {'parameter_server': {'enabled': True}})

tf_estimator.fit('s3://bucket/path/to/training/data')
```

#### 2. Establecer el Global Optimizer

```python
from sagemaker.tensorflow import TensorFlow
# Configure the Training Op (
if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescendOptimizer(learning_rate = 0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step = tf.train.get_global_step())

```


# Incremental Retraining

Se usa para tomar lo aprendido y continuar.

# Ejemplo con Tensorflow 1 y 2

[tensorflow_script_mode_training_and_serving.ipynb](tensorflow_script_mode_training_and_serving.ipynb)